# Sistema especialista manutenção de hardware - Placa mãe

## Instalação do Experta

In [25]:
# Instalação do experta caso n esteja instalado
%pip install experta

Note: you may need to restart the kernel to use updated packages.


## 1 - Definindo os fatos

In [26]:
from experta import * # type: ignore

class PlacaMae(Fact):
    pass

## 2 - Sistema Especialista

As regras foram construídas com base na documentação de fabricantes de BIOS/placas-mãe (AMI e Gigabyte).
Durante o processo de inicialização **(POST – Power-On Self Test)**, a placa-mãe utiliza **códigos sonoros (bipes)** e **sinais visuais/físicos (LEDs, ventoinhas, reinícios em loop, cheiro de queimado)** para indicar falhas de hardware antes mesmo da exibição de mensagens na tela. 

---

### 📌 Regras implementadas
O sistema especialista contempla dois tipos de diagnósticos:

- **Problemas sem bipes** → análise baseada em LEDs, ventoinhas, tela, energia e outros sinais físicos.
- **Problemas com bipes** → análise direta de acordo com tabelas de fabricantes (AMI/Gigabyte).
---

### 📌 Justificativa de cada regra
#### 🔊 Regras com bipes
- **Se o computador emite 1 bipe → Erro de memória RAM**  
  A BIOS emite um bipe curto para indicar falha nos módulos de memória principal.

- **Se o computador emite 2 bipes → Erro de paridade da memória**  
  Indica que a memória não passou no teste de verificação de paridade.

- **Se o computador emite 3 bipes → Erro no endereço de 64 KB da memória básica**  
  Aponta problema em um endereço inicial da memória, fundamental para a inicialização.

- **Se o computador emite 4 bipes → Mau funcionamento do relógio em tempo real (RTC)**  
  O RTC é responsável pelo acompanhamento do tempo e armazenamento de informações de sistema; sua falha impede a configuração correta.

- **Se o computador emite 5 bipes → Erro na CPU**  
  Indica falha no processador, impossibilitando a execução de instruções básicas.

#### 🖥️ Regras sem bipes

- **Se o computador reinicia em loop sem vídeo**  
  Indica falha persistente de inicialização → BIOS corrompida, superaquecimento ou falha de energia.

- **Se nada no computador liga e cheiro de queimado está presente**    
  Indica falha grave → Curto na placa-mãe (capacitores ou chipset queimados).

- **Se o computador liga, dá tela preta, ventoinhas não giram, mas LEDs acendem**  
  Indica que há energia chegando, mas falha na inicialização → Possível problema na placa de vídeo ou cabo de vídeo.

- **Se o computador liga, dá tela preta, ventoinhas giram e LEDs acendem**   
  A energia circula, mas o sistema não inicializa → CPU ou memória RAM com defeito/mal encaixada.

- **Se o computador liga, LEDs piscam, ventoinhas não giram e a está tela preta**  
  Indica falha de inicialização de vídeo/BIOS → Problema na placa de vídeo ou na BIOS.

Esse mapeamento foi implementado diretamente como regras (`@Rule`) no **motor de inferência**, tornando o sistema capaz de simular um diagnóstico automático.


In [27]:
class DiagnosticoPlacaMae(KnowledgeEngine):
    """interpreta a quantidade de bipes ou o sintoma fisico e fornece o diagnóstico correspondente."""
    
    #Possiveis problemas sem bipes
    @Rule(PlacaMae(liga=True, tela_preta=True, bipes=0, ventoinhas=False, leds=True))
    def problema_placa_video(self):
        print("Possível diagnóstico: Problema na placa de vídeo ou cabo de vídeo.")
        
    @Rule(PlacaMae(liga=True, bipes=0, leds=True, ventoinhas=False))
    def leds_sem_ventoinha(self):
        print("Possível diagnóstico: Problema na placa de vídeo ou falha de inicialização da BIOS.")

    @Rule(PlacaMae(liga=True, bipes=0, reinicia=True))
    def reinicia_em_loop(self):
        print("Possível diagnóstico: BIOS corrompida, superaquecimento ou falha de energia.")

    @Rule(PlacaMae(liga=True, tela_preta=True, bipes=0, ventoinhas=True))
    def sem_video_com_ventoinha(self):
        print("Possível diagnóstico: Problema na RAM ou CPU (mal encaixada ou defeituosa).")

    @Rule(PlacaMae(liga=False, bipes=0, cheiro_queimado=True))
    def curto_placa_mae(self):
        print("Possível diagnóstico: Curto na placa-mãe (capacitores ou chipset queimados).")
    
    # Problemas com bipes
    # cod de erros para placa mães Gigabyte e AMI
    @Rule(PlacaMae(bipes=1))
    def erro_memoria(self):
        print("Diagnóstico: Memory Error. Erro de memória")

    @Rule(PlacaMae(bipes=2))
    def erro_paridade_memoria(self):
        print("Diagnóstico: Memory parity check error. Erro de checagem na paridade da memória")

    @Rule(PlacaMae(bipes=3))
    def erro_endereco_memoria(self):
        print("Diagnóstico: basic memory 64K address check error. Erro de checagem do endereço de 64k da memória básica")

    @Rule(PlacaMae(bipes=4))
    def erro_relogio(self):
        print("Diagnóstico: Real Time Clock malfunction. Mau funcionamento do relógio em tempo real")

    @Rule(PlacaMae(bipes=5))
    def erro_cpu(self):
        print("Diagnóstico: CPU error. Erro na CPU")


## 3 - Rodando o Motor de inferencia

### Exemplos de diagnósticos:

In [28]:
engine = DiagnosticoPlacaMae()
engine.reset()

#exemplos sem a placa mãe emitindo bipes
engine.declare(PlacaMae(liga=False, bipes=0, cheiro_queimado=True))
engine.declare(PlacaMae(liga=True, bipes=0, leds=True, ventoinhas=False))

#exemplos com a placa mãe emitindo bipes
engine.declare(PlacaMae(bipes=2))
engine.declare(PlacaMae(bipes=5))

engine.run()
    
  

Diagnóstico: CPU error. Erro na CPU
Diagnóstico: Memory parity check error. Erro de checagem na paridade da memória
Possível diagnóstico: Problema na placa de vídeo ou falha de inicialização da BIOS.
Possível diagnóstico: Curto na placa-mãe (capacitores ou chipset queimados).


### Diagnóstico interativo: 

In [31]:
def perguntar_booleano(pergunta):
    while True:
        resp = input(pergunta + " (s/n): ").strip().lower()
        if resp in ["s", "n"]:
            return resp == "s"
        print("Digite apenas 's' ou 'n'.")

dados = {}

if perguntar_booleano("A placa-mãe está emitindo sons de bipes?"):
    while True:
        try:
            n_bipes = int(input("Digite o número de bipes (1 a 5): "))
            if 1 <= n_bipes <= 5:
                dados["bipes"] = n_bipes
                break
            else:
                print("Número inválido, tente novamente.")
        except ValueError:
            print("Digite um número válido.")
else:
    dados["bipes"] = 0
    dados["liga"] = perguntar_booleano("O computador liga?")
    dados["tela_preta"] = perguntar_booleano("A tela permanece preta?")
    dados["ventoinhas"] = perguntar_booleano("As ventoinhas giram?")
    dados["leds"] = perguntar_booleano("Os LEDs acendem?")
    dados["reinicia"] = perguntar_booleano("O PC reinicia em loop?")
    dados["cheiro_queimado"] = perguntar_booleano("Você percebe cheiro de queimado?")

engine = DiagnosticoPlacaMae()
engine.reset()
engine.declare(PlacaMae(**dados))
engine.run()



Possível diagnóstico: BIOS corrompida, superaquecimento ou falha de energia.
Possível diagnóstico: Problema na RAM ou CPU (mal encaixada ou defeituosa).
